## Building a Text Retrieval Database

### Step 0: Create a connection with your database

In [14]:
import getpass
# This collects a masked password from the user
mypasswd = getpass.getpass()

········


In [15]:
mysso = 'djkgg'
dbname = 'dsa_student'
schema = 'djkgg'

In [16]:
import psycopg2
import numpy as np
from psycopg2.extensions import adapt, register_adapter, AsIs

# Then connects to the DB
connection = psycopg2.connect(database = dbname, 
                              user = mysso, 
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

### Step 1: Create data repository (i.e table) within a database.

```SQL
-------------------------
-- Basic Table 
-------------------------
CREATE TABLE sebcq5.BookLines(
        id SERIAL NOT NULL,
        name varchar(250) NOT NULL,
        line_no INT NOT NULL,
        line text NOT NULL
);

ALTER TABLE sebcq5.BookLines
ADD CONSTRAINT pk_BookLines PRIMARY KEY (id);
```

In [17]:
CREATE_TABLES = f"""
-------------------------
-- Basic Table 
-------------------------
DROP TABLE IF EXISTS djkgg.BookLines;
CREATE TABLE djkgg.BookLines(
        id SERIAL NOT NULL,
        name varchar(250) NOT NULL,
        line_no INT NOT NULL,
        line text NOT NULL
);

ALTER TABLE djkgg.BookLines
ADD CONSTRAINT pk_BookLines PRIMARY KEY (id);
"""
# print(CREATE_TABLES)

In [19]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TABLES)

**Check whether the table is created by listing all the tables in the schema**

In [21]:
SQL_QUERY = f"""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema = 'djkgg';
"""

In [22]:
import pandas as pd

df = pd.read_sql_query(SQL_QUERY, connection)
df

,table_schema,table_name
0,djkgg,f24dmir
1,djkgg,booksearch
2,djkgg,booklines


### Step 2: Add a column that implements the vector model, then parse the data into it.

```SQL
-------------------------
Separate Ts_Vector column
-------------------------
-- TS_Vector for GIN INDEX
ALTER TABLE sebcq5.BookLines
  ADD COLUMN line_tsv_gin tsvector;

UPDATE sebcq5.BookLines
SET line_tsv_gin = to_tsvector('pg_catalog.english', line);
```

In [23]:
UPDATE_TABLE=f"""
-------------------------
-- Separate Ts_Vector column
-------------------------
-- TS_Vector for GIN INDEX
ALTER TABLE djkgg.BookLines
  ADD COLUMN line_tsv_gin tsvector;

UPDATE djkgg.BookLines
SET line_tsv_gin = to_tsvector('pg_catalog.english', line);
"""
# print(UPDATE_TABLE)

In [24]:
with connection, connection.cursor() as cursor:
    cursor.execute(UPDATE_TABLE)

**Check the table**

In [25]:
SQL_QUERY = f"""
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'djkgg' AND table_name = 'booklines';
"""

df = pd.read_sql_query(SQL_QUERY, connection)
df

,table_schema,table_name,column_name,data_type
0,djkgg,booklines,id,integer
1,djkgg,booklines,name,character varying
2,djkgg,booklines,line_no,integer
3,djkgg,booklines,line,text
4,djkgg,booklines,line_tsv_gin,tsvector


### Step 3: Add another column that implements the vector model, then parse the data into it.

```SQL
-- TS_Vector for GIST INDEX
ALTER TABLE sebcq5.BookLines
  ADD COLUMN line_tsv_gist tsvector;

UPDATE sebcq5.BookLines
SET line_tsv_gist = to_tsvector('pg_catalog.english', line);
```

In [26]:
UPDATE_TABLE=f"""
-- TS_Vector for GIST INDEX
ALTER TABLE djkgg.BookLines
  ADD COLUMN line_tsv_gist tsvector;

UPDATE djkgg.BookLines
SET line_tsv_gist = to_tsvector('pg_catalog.english', line);
"""
# print(UPDATE_TABLE)

In [27]:
with connection, connection.cursor() as cursor:
    cursor.execute(UPDATE_TABLE)

**Check the table**

In [28]:
SQL_QUERY = f"""
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'djkgg' AND table_name = 'booklines';
"""

df = pd.read_sql_query(SQL_QUERY, connection)
df

,table_schema,table_name,column_name,data_type
0,djkgg,booklines,id,integer
1,djkgg,booklines,name,character varying
2,djkgg,booklines,line_no,integer
3,djkgg,booklines,line,text
4,djkgg,booklines,line_tsv_gin,tsvector
5,djkgg,booklines,line_tsv_gist,tsvector


### Step 4: Set up database triggers to parse all new content loaded into the vector models.

```SQL
--TRIGGER
CREATE TRIGGER tsv_gin_update 
	BEFORE INSERT OR UPDATE
	ON sebcq5.BookLines 
	FOR EACH ROW 
	EXECUTE PROCEDURE 
	tsvector_update_trigger(line_tsv_gin,'pg_catalog.english',line);

CREATE TRIGGER tsv_gist_update 
	BEFORE INSERT OR UPDATE
	ON sebcq5.BookLines 
	FOR EACH 
	ROW EXECUTE PROCEDURE
	tsvector_update_trigger(line_tsv_gist,'pg_catalog.english',line);

```

In [29]:
CREATE_TRIGGER=f"""
--TRIGGER
CREATE TRIGGER tsv_gin_update 
    BEFORE INSERT OR UPDATE
    ON djkgg.BookLines 
    FOR EACH ROW 
    EXECUTE PROCEDURE 
    tsvector_update_trigger(line_tsv_gin,'pg_catalog.english',line);

CREATE TRIGGER tsv_gist_update 
    BEFORE INSERT OR UPDATE
    ON djkgg.BookLines 
    FOR EACH 
    ROW EXECUTE PROCEDURE
    tsvector_update_trigger(line_tsv_gist,'pg_catalog.english',line);

"""

In [30]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TRIGGER)

### Step 5:  Add a specialized indexing to the vector models.

```SQL
-------------------------
-- Create Indexes
-------------------------

-- Index on content (Trigram needed,to use Gin Index)
-- CREATE EXTENSION pg_trgm;  -- Done by DB Admin

CREATE INDEX BookLines_line
ON sebcq5.BookLines USING GIN(line gin_trgm_ops);

-- GIN INDEX on content_tsv_gin
CREATE INDEX BookLines_line_tsv_gin
ON sebcq5.BookLines USING GIN(line_tsv_gin);

-- GIST INDEX on content_tsv_gist
CREATE INDEX BookLines_line_tsv_gist
ON sebcq5.BookLines USING GIST(line_tsv_gist);
```

In [31]:
CREATE_INDEX=f"""
-------------------------
-- Create Indexes
-------------------------

-- Index on content (Trigram needed,to use Gin Index)
-- CREATE EXTENSION pg_trgm;  -- Done by DB Admin

CREATE INDEX BookLines_line
ON djkgg.BookLines USING GIN(line gin_trgm_ops);

-- GIN INDEX on content_tsv_gin
CREATE INDEX BookLines_line_tsv_gin
ON djkgg.BookLines USING GIN(line_tsv_gin);

-- GIST INDEX on content_tsv_gist
CREATE INDEX BookLines_line_tsv_gist
ON djkgg.BookLines USING GIST(line_tsv_gist);
"""

In [32]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_INDEX)

### Grant permission to read your table
```SQL
GRANT USAGE ON SCHEMA djkgg TO dsa_ro_user;
GRANT SELECT ON djkgg.BookLines TO dsa_ro_user;
```

In [36]:
GRANT_ACCESS=f"""
GRANT USAGE ON SCHEMA djkgg TO dsa_ro_user;
GRANT SELECT ON djkgg.booklines TO dsa_ro_user;
"""

In [37]:
with connection, connection.cursor() as cursor:
    cursor.execute(GRANT_ACCESS)